In [ ]:
import pandas as pd

In [ ]:
singlerp = pd.read_csv("singlerp.csv")
date = "prdate_parsed"

In [ ]:
singlerp = pd.read_csv("df_pathology_biopsy_final.csv")
date = "Report_Date_Time"

In [ ]:
psa = pd.read_csv("labs_cl.csv")

In [ ]:
psa = psa[psa['Percent Free'] == False]

In [ ]:
psa['time_lab_result'] = pd.to_datetime(psa['time_lab_result'], infer_datetime_format=True) 
singlerp[date] = pd.to_datetime(singlerp[date], infer_datetime_format=True)

In [ ]:
recurrence = pd.DataFrame(columns=['EMPI', 'num_before_rp', 'detectable_postop_psa', 'bcr_rp', 'bcr_date', 'first_postop_psa','preop_psa', 'postop_psas', 'preop_psa_date','postop_psas_dates', 'prdate_parsed'])

In [ ]:
import datetime

k = 0
for i, row in singlerp.iterrows():
    psa_records = psa[psa['EMPI'] == row['EMPI']]
    if len(psa_records) > 0:
        sorted_records = psa_records.sort_values(by='time_lab_result', ascending=True)
        sorted_records = sorted_records[~sorted_records['time_lab_result'].isnull()]
        row_num = 0
        preop_psa = float("nan")
        preop_psa_date = float("nan")
        for j, row2 in sorted_records.iterrows():
            if row2['time_lab_result'] < row[date]:
                preop_psa = row2['PSA']
                preop_psa_date = row2['time_lab_result'].date()
            if row2['time_lab_result'] >= row[date] + datetime.timedelta(days=14):
                months = (row2['time_lab_result'].year - row[date].year) * 12 + (row2['time_lab_result'].month - row[date].month)
                if months <= 6:
                    if row2['undetect'] or row2['PSA'] <= 0.10:
                        detectable_postop_psa = 0
                    else:
                        detectable_postop_psa = 1
                else:
                    detectable_postop_psa = float("nan")
                    
                if detectable_postop_psa == 0:
                    post_surgery_psa = sorted_records[sorted_records['time_lab_result'] >= row[date]  + datetime.timedelta(days=14)]
                    post_surgery_psa = post_surgery_psa[post_surgery_psa['PSA'] > 0.2]
                    if len(post_surgery_psa) >= 2:
                        bcr_rp = 1
                        bcr_date = post_surgery_psa.time_lab_result.iloc[1]
                    else:
                        bcr_rp = 0
                        bcr_date = sorted_records.time_lab_result.iloc[-1]
                    
                else:
                    bcr_rp = float("nan")
                    bcr_date = float("nan")
                
                post_surgery_psa = sorted_records[sorted_records['time_lab_result'] >= row[date]  + datetime.timedelta(days=14)]
                first_postop_psa = post_surgery_psa.time_lab_result.iloc[0]
                postop_psas = list(post_surgery_psa['PSA'])
                
                postop_psas_dates = list(post_surgery_psa['time_lab_result'].apply(lambda row: row.date()))
                
                recurrence.loc[k] = [row['EMPI'], row_num, detectable_postop_psa, bcr_rp, bcr_date, first_postop_psa, preop_psa, postop_psas, preop_psa_date, postop_psas_dates, row[date]]
                k += 1
                break
            
            
            row_num += 1

In [ ]:
len(recurrence[~recurrence['preop_psa'].isnull()])

In [ ]:
len(recurrence[recurrence['preop_psa'].isnull()])

In [ ]:
recurrence_output = recurrence[['EMPI','detectable_postop_psa','bcr_rp','bcr_date']]
recurrence_output.to_csv("outcome.csv", na_rep='NaN')

In [ ]:
validation = recurrence[['EMPI', 'preop_psa', 'prdate_parsed','postop_psas', 'preop_psa_date','postop_psas_dates','bcr_date','bcr_rp']]
validation.to_csv("validation.csv", na_rep='NaN')

In [ ]:
recurrence['detectable_postop_psa'].value_counts(normalize=True) * 100

In [ ]:
pathology = pd.read_csv('pathology_rp_all_feats_final_v2.tsv', sep='\t')

In [ ]:
#merge recurrence data with pathology patients
recurrence_merged = pd.merge(pathology, recurrence, on='EMPI')

In [ ]:
recurrence_merged['num_before_rp'].median()

In [ ]:
recurrence_merged['num_before_rp'].mean()

In [ ]:
recurrence['num_before_rp'].mean()